## Use BentoML with ONNX model zoo(resnet50)

This example notebook demonstrates how to use ONNX model zoo with BentoML.  It defines a BentoService with `resnet50` model and deploys it to AWS sagemaker as an API endpoint.

original notebook: https://github.com/onnx/onnx-docker/blob/master/onnx-ecosystem/inference_demos/resnet50_modelzoo_onnxruntime_inference.ipynb

In [5]:
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime
import onnx
from onnx import numpy_helper
import urllib.request
import json
import time

# display images in notebook
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

%matplotlib inline

In [ ]:
onnx_model_url = "https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.tar.gz"
imagenet_labels_url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"

# retrieve our model from the ONNX model zoo
urllib.request.urlretrieve(onnx_model_url, filename="resnet50v2.tar.gz")
urllib.request.urlretrieve(imagenet_labels_url, filename="imagenet-simple-labels.json")

!curl https://raw.githubusercontent.com/onnx/onnx-docker/master/onnx-ecosystem/inference_demos/images/dog.jpg -o dog.jpg
!tar xvzf resnet50v2.tar.gz

### Load sample outputs and inputs

In [ ]:
test_data_dir = 'resnet50v2/test_data_set'
test_data_num = 3

In [ ]:
import glob
import os

# Load inputs
inputs = []
for i in range(test_data_num):
    input_file = os.path.join(test_data_dir + '_{}'.format(i), 'input_0.pb')
    tensor = onnx.TensorProto()
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
        inputs.append(numpy_helper.to_array(tensor))

print('Loaded {} inputs successfully.'.format(test_data_num))
        
# Load reference outputs

ref_outputs = []
for i in range(test_data_num):
    output_file = os.path.join(test_data_dir + '_{}'.format(i), 'output_0.pb')
    tensor = onnx.TensorProto()
    with open(output_file, 'rb') as f:
        tensor.ParseFromString(f.read())    
        ref_outputs.append(numpy_helper.to_array(tensor))
        
print('Loaded {} reference outputs successfully.'.format(test_data_num))

In [4]:
def load_labels(path):
    with open(path) as f:
        data = json.load(f)
    return np.asarray(data)

labels = load_labels('imagenet-simple-labels.json')
labels

array(['tench', 'goldfish', 'great white shark', 'tiger shark',
       'hammerhead shark', 'electric ray', 'stingray', 'cock', 'hen',
       'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco',
       'indigo bunting', 'American robin', 'bulbul', 'jay', 'magpie',
       'chickadee', 'American dipper', 'kite', 'bald eagle', 'vulture',
       'great grey owl', 'fire salamander', 'smooth newt', 'newt',
       'spotted salamander', 'axolotl', 'American bullfrog', 'tree frog',
       'tailed frog', 'loggerhead sea turtle', 'leatherback sea turtle',
       'mud turtle', 'terrapin', 'box turtle', 'banded gecko',
       'green iguana', 'Carolina anole',
       'desert grassland whiptail lizard', 'agama',
       'frilled-necked lizard', 'alligator lizard', 'Gila monster',
       'European green lizard', 'chameleon', 'Komodo dragon',
       'Nile crocodile', 'American alligator', 'triceratops',
       'worm snake', 'ring-necked snake', 'eastern hog-nosed snake',
       'smooth green snak

In [5]:
%%writefile onnx_resnet50.py

import numpy as np

import bentoml
from bentoml.artifact import OnnxModelArtifact, PickleArtifact
from bentoml.handlers import ImageHandler


@bentoml.env(auto_pip_dependencies=True)
@bentoml.artifacts([OnnxModelArtifact('model'), PickleArtifact('labels')])
class OnnxResnet50(bentoml.BentoService):
    def preprocess(self, input_data):
        # convert the input data into the float32 input
        img_data = input_data.transpose(2, 0, 1).astype('float32')

        #normalize
        mean_vec = np.array([0.485, 0.456, 0.406])
        stddev_vec = np.array([0.229, 0.224, 0.225])
        norm_img_data = np.zeros(img_data.shape).astype('float32')
        for i in range(img_data.shape[0]):
            norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
        
        #add batch channel
        norm_img_data = norm_img_data.reshape(1, 3, 224, 224).astype('float32')
        return norm_img_data
    
    def softmax(self, x):
        x = x.reshape(-1)
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
    
    def post_process(self, raw_result):
        return self.softmax(np.array(raw_result)).tolist()

 
    @bentoml.api(ImageHandler)
    def predict(self, image_data):
        input_data = self.preprocess(image_data)
        input_name = self.artifacts.model.get_inputs()[0].name
        raw_result = self.artifacts.model.run([], {input_name: input_data})
        result = self.post_process(raw_result)
        idx = np.argmax(result)
        sort_idx = np.flip(np.squeeze(np.argsort(result)))
        
        # return top 5 labels
        return self.artifacts.labels[sort_idx[:5]]
        


Overwriting onnx_resnet50.py


In [ ]:
from onnx_resnet50 import OnnxResnet50

svc = OnnxResnet50()
svc.pack('labels', labels)
svc.pack('model', 'resnet50v2/resnet50v2.onnx')

saved_path = svc.save()
saved_path

[2020-05-27 15:48:25,947] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-27 15:48:26,279] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.7.8+8.g26d1dc0
creating BentoML-0.7.8+8.g26d1dc0/BentoML.egg-info
creating BentoML-0.7.8+8.g26d1dc0/bentoml
creating BentoML-0.7.8+8.g26d1dc0/bentoml/artifact
creating BentoML-0.7.8+8.g26d1dc0/bentoml/bundler
creating BentoML-0.7.8+8.g26d1dc0/bentoml/cli
creating BentoML-0.7.8+8.g26d1dc0/bentoml/clipper
creating BentoML-0.7.8+8.g26d1dc0/bentoml/configuration
creating BentoML-0.7.8+8.g26d1dc0/bentoml/configuration/__pycache__
creating BentoML-0.7.8+8.g26d1dc0/bentoml/deployment
creating BentoML-0.7.8+8.g26d1dc0/bentoml/deployment/aws_lambda
creating BentoML-0.7.8+8.g26d1dc0/bentoml/deployment/sagemaker
creating BentoML-0.7.8+8.g26d1dc0/bentoml/handlers
creating BentoML-0.7.8+8.g26d1dc0/bentoml/marshal
creating BentoML-0.7.8+8.g26d1dc0/bentoml/migrations
creating BentoML-0.7.8+8.g26d1dc0/bentoml/migrations/__pycache__
creating BentoML-0.7.8+8.g26d1dc0/bentoml/migrations/versions
creating BentoML-0.7.8+8

creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/negotiator/lib
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/on-finished
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/parseurl
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/path-to-regexp
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/protobufjs
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/protobufjs/bin
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/protobufjs/cli
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/protobufjs/cli/bin
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/protobufjs/cli/lib
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/protobufjs/cli/lib/tsd-jsdoc
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/protobufjs/cli/targets
creating BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/

copying bentoml/clipper/__init__.py -> BentoML-0.7.8+8.g26d1dc0/bentoml/clipper
copying bentoml/configuration/__init__.py -> BentoML-0.7.8+8.g26d1dc0/bentoml/configuration
copying bentoml/configuration/__init__.pyc -> BentoML-0.7.8+8.g26d1dc0/bentoml/configuration
copying bentoml/configuration/configparser.py -> BentoML-0.7.8+8.g26d1dc0/bentoml/configuration
copying bentoml/configuration/configparser.pyc -> BentoML-0.7.8+8.g26d1dc0/bentoml/configuration
copying bentoml/configuration/default_bentoml.cfg -> BentoML-0.7.8+8.g26d1dc0/bentoml/configuration
copying bentoml/configuration/__pycache__/__init__.cpython-36.pyc -> BentoML-0.7.8+8.g26d1dc0/bentoml/configuration/__pycache__
copying bentoml/configuration/__pycache__/__init__.cpython-37.pyc -> BentoML-0.7.8+8.g26d1dc0/bentoml/configuration/__pycache__
copying bentoml/configuration/__pycache__/configparser.cpython-36.pyc -> BentoML-0.7.8+8.g26d1dc0/bentoml/configuration/__pycache__
copying bentoml/configuration/__pycache__/configparser

...

copying bentoml/yatai/web/dist/node_modules/@types/node/globals.d.ts -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/@types/node
copying bentoml/yatai/web/dist/node_modules/@types/node/http.d.ts -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/@types/node
copying bentoml/yatai/web/dist/node_modules/@types/node/http2.d.ts -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/@types/node
copying bentoml/yatai/web/dist/node_modules/@types/node/https.d.ts -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/@types/node
copying bentoml/yatai/web/dist/node_modules/@types/node/index.d.ts -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/@types/node
copying bentoml/yatai/web/dist/node_modules/@types/node/inspector.d.ts -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/@types/node
copying bentoml/yatai/web/dist/node_modules/@types/node/module.d.ts -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/@type

copying bentoml/yatai/web/dist/node_modules/bytes/Readme.md -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/bytes
copying bentoml/yatai/web/dist/node_modules/bytes/index.js -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/bytes
copying bentoml/yatai/web/dist/node_modules/bytes/package.json -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/bytes
copying bentoml/yatai/web/dist/node_modules/content-disposition/HISTORY.md -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/content-disposition
copying bentoml/yatai/web/dist/node_modules/content-disposition/LICENSE -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/content-disposition
copying bentoml/yatai/web/dist/node_modules/content-disposition/README.md -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/content-disposition
copying bentoml/yatai/web/dist/node_modules/content-disposition/index.js -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_module

copying bentoml/yatai/web/dist/node_modules/encodeurl/README.md -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/encodeurl
copying bentoml/yatai/web/dist/node_modules/encodeurl/index.js -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/encodeurl
copying bentoml/yatai/web/dist/node_modules/encodeurl/package.json -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/encodeurl
copying bentoml/yatai/web/dist/node_modules/escape-html/LICENSE -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/escape-html
copying bentoml/yatai/web/dist/node_modules/escape-html/Readme.md -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/escape-html
copying bentoml/yatai/web/dist/node_modules/escape-html/index.js -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/escape-html
copying bentoml/yatai/web/dist/node_modules/escape-html/package.json -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/escape-html
copying bentoml/

copying bentoml/yatai/web/dist/node_modules/iconv-lite/encodings/sbcs-data.js -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/iconv-lite/encodings
copying bentoml/yatai/web/dist/node_modules/iconv-lite/encodings/utf16.js -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/iconv-lite/encodings
copying bentoml/yatai/web/dist/node_modules/iconv-lite/encodings/utf7.js -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/iconv-lite/encodings
copying bentoml/yatai/web/dist/node_modules/iconv-lite/encodings/tables/big5-added.json -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/iconv-lite/encodings/tables
copying bentoml/yatai/web/dist/node_modules/iconv-lite/encodings/tables/cp936.json -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/iconv-lite/encodings/tables
copying bentoml/yatai/web/dist/node_modules/iconv-lite/encodings/tables/cp949.json -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/iconv-lite/encodin

copying bentoml/yatai/web/dist/node_modules/mime-db/HISTORY.md -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/mime-db
copying bentoml/yatai/web/dist/node_modules/mime-db/LICENSE -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/mime-db
copying bentoml/yatai/web/dist/node_modules/mime-db/README.md -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/mime-db
copying bentoml/yatai/web/dist/node_modules/mime-db/db.json -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/mime-db
copying bentoml/yatai/web/dist/node_modules/mime-db/index.js -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/mime-db
copying bentoml/yatai/web/dist/node_modules/mime-db/package.json -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/mime-db
copying bentoml/yatai/web/dist/node_modules/mime-types/HISTORY.md -> BentoML-0.7.8+8.g26d1dc0/bentoml/yatai/web/dist/node_modules/mime-types
copying bentoml/yatai/web/dist/node_modules/mime-types/L

In [6]:
image = Image.open('dog.jpg')

In [ ]:
image_data = np.array(image).transpose(2, 0, 1)
svc.predict(image_data)

In [3]:
!bentoml get OnnxResnet50

BENTO_SERVICE                       AGE                           APIS                   ARTIFACTS
OnnxResnet50:20200527154826_2F88FF  3 minutes and 39.33 seconds   predict<ImageHandler>  labels<PickleArtifact>, model<OnnxModelArtifact>
OnnxResnet50:20200527154509_967CC2  6 minutes and 55.15 seconds   predict<ImageHandler>  labels<PickleArtifact>, model<OnnxModelArtifact>
OnnxResnet50:20200527154049_855E2E  11 minutes and 16.17 seconds  predict<ImageHandler>  labels<PickleArtifact>, model<OnnxModelArtifact>
OnnxResnet50:20200527153457_857027  17 minutes and 5.76 seconds   predict<ImageHandler>  labels<PickleArtifact>, model<OnnxModelArtifact>


In [4]:
!bentoml run OnnxResnet50:latest predict --input dog.jpg

[2020-05-27 15:52:21,569] INFO - Getting latest version OnnxResnet50:20200527154826_2F88FF
[2020-05-27 15:52:21,570] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-27 15:52:21,586] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.8,  but loading from BentoML version 0.7.8+8.g26d1dc0
[2020-05-27 15:52:21,609] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled 

## Model Serving via REST API

In your termnial, run the following command to start the REST API server:

### Run REST API server

In [18]:
!bentoml serve OnnxResnet50:latest

[2020-05-27 16:53:16,984] INFO - Getting latest version OnnxResnet50:20200527154826_2F88FF
[2020-05-27 16:53:16,985] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-27 16:53:17,001] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.8,  but loading from BentoML version 0.7.8+5.gfcc3b38
[2020-05-27 16:53:17,048] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled 

Sending POST request from termnial:
```bash
curl -X POST "http://127.0.0.1:5000/predict" -F image=@dog.jpg
```

```bash
curl -X POST "http://127.0.0.1:5000/predict" -H "Content-Type: image/png" --data-binary @dog.jpg
```

Go visit http://127.0.0.1:5000/ from your browser, click `/predict` -> `Try it out` -> `Choose File` -> `Execute` to sumbit an image from your computer

### Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this project, we are going to deploy to AWS sagemaker

`bentoml sagemaker deploy` command deploys to AWS sagemaker with a single command

In [13]:
!bentoml sagemaker deploy onnx-resnet50 -b OnnxResnet50:20200527154826_2F88FF --api-name predict

Deploying Sagemaker deployment \we in here
root in create archive /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-r1s4by4i/OnnxResnet50
/docker build <tempfile._TemporaryFileWrapper object at 0x7f9ccdb08b00> {'t': '192023623294.dkr.ecr.us-west-2.amazonaws.com/onnxresnet50-sagemaker:20200527154826_2F88FF', 'remote': None, 'q': False, 'nocache': False, 'rm': False, 'forcerm': False, 'pull': False, 'dockerfile': ('Dockerfile-sagemaker', None)}
\docker response <Response [200]>
context closes
print responses
[2020-05-27 15:56:36,296] INFO - Step 1/11 : FROM continuumio/miniconda3:4.8.2
[2020-05-27 15:56:36,296] INFO - 

\[2020-05-27 15:56:54,522] INFO -  ---> b4adc22212f1

[2020-05-27 15:56:54,522] INFO - Step 2/11 : EXPOSE 8080
[2020-05-27 15:56:54,522] INFO - 

|[2020-05-27 15:56:54,893] INFO -  ---> Running in 4cb4b48824ca

-[2020-05-27 15:56:55,022] INFO -  ---> 58b87040bb30

[2020-05-27 15:56:55,022] INFO - Step 3/11 : RUN set -x      && apt-get update      

|[2020-05-27 15:57:02,164] INFO - debconf: delaying package configuration, since apt-utils is not installed

\[2020-05-27 15:57:02,200] INFO - Fetched 47.2 MB in 2s (24.3 MB/s)

[2020-05-27 15:57:02,222] INFO - Selecting previously unselected package binutils-common:amd64.
(Reading database ... 
(Reading database ... 75% INFO - (Reading database ... 5%
[2020-05-27 15:57:02,228] INFO - (Reading database ... 80%
[2020-05-27 15:57:02,229] INFO - (Reading database ... 85%
[2020-05-27 15:57:02,231] INFO - (Reading database ... 90%
[2020-05-27 15:57:02,233] INFO - (Reading database ... 95%
(Reading database ... 12564 files and directories currently installed.)

[2020-05-27 15:57:02,236] INFO - Preparing to unpack .../00-binutils-common_2.31.1-16_amd64.deb ...

[2020-05-27 15:57:02,239] INFO - Unpacking binutils-common:amd64 (2.31.1-16) ...

|[2020-05-27 15:57:02,538] INFO - Selecting previously unselected package libbinutils:amd64.

[2020-05-27 15:57:02,540] INFO - Preparing to unpack ...

-[2020-05-27 15:57:08,839] INFO - Selecting previously unselected package dpkg-dev.

[2020-05-27 15:57:08,841] INFO - Preparing to unpack .../30-dpkg-dev_1.19.7_all.deb ...

[2020-05-27 15:57:08,844] INFO - Unpacking dpkg-dev (1.19.7) ...

/[2020-05-27 15:57:08,950] INFO - Selecting previously unselected package build-essential.

[2020-05-27 15:57:08,953] INFO - Preparing to unpack .../31-build-essential_12.6_amd64.deb ...

[2020-05-27 15:57:08,956] INFO - Unpacking build-essential (12.6) ...

[2020-05-27 15:57:08,991] INFO - Selecting previously unselected package libpq5:amd64.

[2020-05-27 15:57:08,993] INFO - Preparing to unpack .../32-libpq5_11.7-0+deb10u1_amd64.deb ...

[2020-05-27 15:57:08,996] INFO - Unpacking libpq5:amd64 (11.7-0+deb10u1) ...

|[2020-05-27 15:57:09,044] INFO - Selecting previously unselected package libpq-dev.

[2020-05-27 15:57:09,047] INFO - Preparing to unpack .../33-libpq-dev_11.7-0+deb10u1_amd64.deb ...

[2020-05-27 15:57:09,050] INFO - Unpacking libpq-

/[2020-05-27 15:57:11,794] INFO - Get:27 http://deb.debian.org/debian buster/main amd64 nginx all 1.14.2-2+deb10u1 [88.3 kB]

\[2020-05-27 15:57:12,020] INFO - debconf: delaying package configuration, since apt-utils is not installed

[2020-05-27 15:57:12,055] INFO - Fetched 7787 kB in 1s (14.0 MB/s)

[2020-05-27 15:57:12,080] INFO - Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 
(Reading database ... 75%1] INFO - (Reading database ... 5%
[2020-05-27 15:57:12,090] INFO - (Reading database ... 80%
[2020-05-27 15:57:12,109] INFO - (Reading database ... 85%
[2020-05-27 15:57:12,122] INFO - (Reading database ... 90%
[2020-05-27 15:57:12,125] INFO - (Reading database ... 95%
(Reading database ... 16014 files and directories currently installed.)

[2020-05-27 15:57:12,128] INFO - Preparing to unpack .../00-fonts-dejavu-core_2.37-1_all.deb ...

[2020-05-27 15:57:12,131] INFO - Unpacking fonts-dejavu-core (2.37-1) ...

\[2020-05-27 15:57:12,424] INFO - Sel

[2020-05-27 15:57:14,519] INFO - Unpacking nginx (1.14.2-2+deb10u1) ...

-[2020-05-27 15:57:14,566] INFO - Setting up libxpm4:amd64 (1:3.5.12-1) ...

[2020-05-27 15:57:14,577] INFO - Setting up nginx-common (1.14.2-2+deb10u1) ...

|[2020-05-27 15:57:14,743] INFO - debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline

-[2020-05-27 15:57:15,024] INFO - Setting up libjbig0:amd64 (2.1-3.1+b2) ...

[2020-05-27 15:57:15,034] INFO - Setting up libjpeg62-turbo:amd64 (1:1.5.2-2+b1) ...

/[2020-05-27 15:57:15,042] INFO - Setting up libnginx-mod-http-auth-pam (1.14.2-2+deb10u1) ...

[2020-05-27 15:57:15,072] INFO - Setting up libpng16-16:amd64 (1.6.36-6) ...

[2020-05-27 15:57:15,082] INFO - Setting up libwebp6:amd64 (0.6.1-2) ...

[2020-05-27 15:57:15,091] INFO - Setting up fonts-dejavu-core (2.37-1) ...

|[2020-05-27 15:57:15,152] INFO - Setting up libnginx-mod-http-echo (1.14.2-2+deb10u

mkl-2020.1           | 129.0 MB  | 3          |   3% 
mkl-2020.1           | 129.0 MB  | 5          |   6% 
mkl-2020.1           | 129.0 MB  | 7          |   8% 
mkl-2020.1           | 129.0 MB  | #          |  10% 
mkl-2020.1           | 129.0 MB  | #2         |  12% 
mkl-2020.1           | 129.0 MB  | #4         |  15% 
mkl-2020.1           | 129.0 MB  | #6         |  17% 
mkl-2020.1           | 129.0 MB  | #8         |  19% 
mkl-2020.1           | 129.0 MB  | ##1        |  21% 
mkl-2020.1           | 129.0 MB  | ##3        |  24% 
mkl-2020.1           | 129.0 MB  | ##5        |  26% 
mkl-2020.1           | 129.0 MB  | ##7        |  28% 
mkl-2020.1           | 129.0 MB  | ###        |  30% 
mkl-2020.1           | 129.0 MB  | ###2       |  33% 
mkl-2020.1           | 129.0 MB  | ###4       |  35% 
mkl-2020.1           | 129.0 MB  | ###7       |  37% 
mkl-2020.1           | 129.0 MB  | ###9       |  39% 
mkl-2020.1           | 129.0 MB  | ####1      |  41% 
mkl-2020.1           | 129.0

\[2020-05-27 15:58:37,780] INFO -   Downloading BentoML-0.7.8-py3-none-any.whl (3.0 MB)

/[2020-05-27 15:58:38,871] INFO - Collecting numpy==1.18.4

|[2020-05-27 15:58:38,900] INFO -   Downloading numpy-1.18.4-cp37-cp37m-manylinux1_x86_64.whl (20.2 MB)

-[2020-05-27 15:58:41,224] INFO - Collecting onnx>=1.2.3

/[2020-05-27 15:58:41,249] INFO -   Downloading onnx-1.7.0-cp37-cp37m-manylinux1_x86_64.whl (7.4 MB)

/[2020-05-27 15:58:42,883] INFO - Collecting protobuf

[2020-05-27 15:58:42,913] INFO -   Downloading protobuf-3.12.2-cp37-cp37m-manylinux1_x86_64.whl (1.3 MB)

-[2020-05-27 15:58:43,675] INFO - Collecting pillow

/[2020-05-27 15:58:43,704] INFO -   Downloading Pillow-7.1.2-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)

/[2020-05-27 15:58:44,558] INFO - Collecting aiohttp

[2020-05-27 15:58:44,591] INFO -   Downloading aiohttp-3.6.2-cp37-cp37m-manylinux1_x86_64.whl (1.2 MB)

-[2020-05-27 15:58:44,919] INFO - Collecting sqlalchemy>=1.3.0

/[2020-05-27 15:58:44,954] INFO -  

[2020-05-27 15:59:01,195] INFO -   Downloading jmespath-0.10.0-py2.py3-none-any.whl (24 kB)

/[2020-05-27 15:59:01,292] INFO - Collecting websocket-client>=0.32.0

[2020-05-27 15:59:01,325] INFO -   Downloading websocket_client-0.57.0-py2.py3-none-any.whl (200 kB)

|[2020-05-27 15:59:01,405] INFO - Collecting thriftpy2>=0.4.0

[2020-05-27 15:59:01,438] INFO -   Downloading thriftpy2-0.4.11.tar.gz (498 kB)

/[2020-05-27 15:59:02,119] INFO - Collecting pytz>=2017.2

[2020-05-27 15:59:02,149] INFO -   Downloading pytz-2020.1-py2.py3-none-any.whl (510 kB)

\[2020-05-27 15:59:02,387] INFO - Collecting Mako

-[2020-05-27 15:59:02,414] INFO -   Downloading Mako-1.1.2-py2.py3-none-any.whl (75 kB)

[2020-05-27 15:59:02,495] INFO - Collecting python-editor>=0.3

/[2020-05-27 15:59:02,516] INFO -   Downloading python_editor-1.0.4-py3-none-any.whl (4.9 kB)

|[2020-05-27 15:59:02,600] INFO - Collecting MarkupSafe>=0.23

[2020-05-27 15:59:02,624] INFO -   Downloading MarkupSafe-1.1.1-cp37-cp3

In [14]:
!bentoml sagemaker get onnx-resnet50

{
  "namespace": "dev",
  "name": "onnx-resnet50",
  "spec": {
    "bentoName": "OnnxResnet50",
    "bentoVersion": "20200527154826_2F88FF",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict",
      "timeout": 60
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "dev-onnx-resnet50",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-onnx-resnet50",
      "EndpointConfigName": "dev-onnx-resnet50-OnnxResnet50-20200527154826-2F88FF",
      "ProductionVariants": [
        {
          "VariantName": "dev-onnx-resnet50-OnnxResnet50-20200527154826-2F88FF",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/onnxresnet50-sagemaker:20200527154826_2F88FF",
              "ResolvedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/

Test and validate Sagemaker deployment with `aws sagemaker-runtime invoke-endpoint` command

In [17]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name dev-onnx-resnet50 \
--body fileb:///Users/bozhaoyu/src/bento_gallery/onnx/model-zoo/dog.jpg \
--content-type image/jpg output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "dev-onnx-resnet50-OnnxResnet50-20200527154826-2F88FF"
}
["Golden Retriever", "Labrador Retriever", "Sussex Spaniel", "Vizsla", "Otterhound"]

**Clean up Sagemaker deployment with `bentoml sagemaker delete`**

In [12]:
!bentoml sagemaker delete onnx-resnet50

Successfully deleted AWS Lambda deployment "onnx-resnet50"
